In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymysql


# Connexion à MySQL
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='Baz9oula<3',
    database='business_management',
    port=3306
)

# Chargement des données depuis MySQL
products = pd.read_sql("SELECT * FROM Products", connection)
customers = pd.read_sql("SELECT * FROM Customers", connection)
orders = pd.read_sql("SELECT * FROM `Order`", connection)
order_items = pd.read_sql("SELECT * FROM OrderItem", connection)

# Conversion des types numériques (MySQL Decimal -> float)
products["price"] = pd.to_numeric(products["price"], errors="coerce")
products["quantity_in_stock"] = pd.to_numeric(products["quantity_in_stock"], errors="coerce")
order_items["quantity"] = pd.to_numeric(order_items["quantity"], errors="coerce")

print("Données chargées depuis MySQL avec succès")


# Fusion des tables 
data = order_items.merge(orders, on="orderitem_id") \
                  .merge(products, on="product_id") \
                  .merge(customers, on="customer_id", suffixes=("_product", "_customer"))

print("Tables fusionnées")

# Conversion date
data["order_date"] = pd.to_datetime(data["order_date"])

# Création du mois
data["month"] = data["order_date"].dt.month

# Calcul du chiffre d'affaires
data["revenue"] = data["quantity"] * data["price"]


# Chiffre d'affaires par mois
revenue_per_month = data.groupby("month")["revenue"].sum()
print("\n Chiffre d'affaires par mois")
print(revenue_per_month)

# Produits les plus vendus
best_products = data.groupby("last_name_product")["quantity"].sum().sort_values(ascending=False)
print("\n Produits les plus vendus")
print(best_products)

# Valeur totale du stock
products["stock_value"] = products["price"] * products["quantity_in_stock"]
total_stock_value = products["stock_value"].sum()
print("\n Valeur totale du stock :", total_stock_value)

# Valeur moyenne des commandes
order_total = data.groupby("order_id")["revenue"].sum()
average_order_value = np.mean(order_total)
print("\n Valeur moyenne d'une commande :", average_order_value)

# Fréquence d'achat par client
customer_frequency = data.groupby("customer_id")["order_id"].nunique()
print("\n Fréquence d'achat par client")
print(customer_frequency)

# Visualisation 
if not revenue_per_month.empty:
    revenue_per_month.plot(kind="bar")
    plt.title("Chiffre d'affaires par mois")
    plt.xlabel("Mois")
    plt.ylabel("Revenu")
    plt.show()
else:
    print("Pas de données de revenus à afficher.")

# Fermer la connexion
connection.close()

Données chargées depuis MySQL avec succès
Tables fusionnées

 Chiffre d'affaires par mois
Series([], Name: revenue, dtype: int64)

 Produits les plus vendus
Series([], Name: quantity, dtype: int64)

 Valeur totale du stock : 0

 Valeur moyenne d'une commande : nan

 Fréquence d'achat par client
Series([], Name: order_id, dtype: int64)
Pas de données de revenus à afficher.


C:\Users\frago\AppData\Local\Temp\ipykernel_26300\3466705401.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products = pd.read_sql("SELECT * FROM Products", connection)
C:\Users\frago\AppData\Local\Temp\ipykernel_26300\3466705401.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers = pd.read_sql("SELECT * FROM Customers", connection)
C:\Users\frago\AppData\Local\Temp\ipykernel_26300\3466705401.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders = pd.read_sql("SELECT * FROM `Order`", connection)
C:\Us